In [34]:
import requests

base_url = 'http://localhost:1803'

api_base_url = base_url + '/api/v1'
flights_url = api_base_url + '/flights'
bookings_url = api_base_url + '/bookings'
aircrafts_url = api_base_url + '/aircrafts'
users_url = api_base_url + '/users'
passengers_url = api_base_url + '/passengers'

s = requests.Session()
citizen_id = 1

In [45]:
aircraft = {
    "aircraft_id": 1,
    "aircraft_manufacturer": "boeing",
    "aircraft_name": "747",
    "aircraft_type": "passenger",
    "capacity": 500,
}

flight = {
    "aircraft_id": 1,
    "arrival_airport": "CXR",
    "arrival_time": "2024-12-12T02:10:20+07:00",
    "board_time": "2024-12-12T02:10:20+07:00",
    "departure_airport": "HAN",
    "departure_time": "2024-12-12T02:10:20+07:00",
    "flight_id": 0,
    "flight_number": "VN123",
    "gate": "A1",
    "status": "on-time",
}

booking = {
    "booking_date": "2024-12-12T02:10:20+07:00",
    "booking_id": 0,
    "citizen_id": "0123",
    "class": "business",
    "flight_id": 1,
    "price": 20200.14,
    "seat_number": "string",
    "total_passengers": 20,
}

user = {
    "citizen_id": f"{citizen_id}",
    "dob": "2024-12-12T02:10:20+07:00",
    "email": {
        "string": "string",
        "valid": True,
    },
    "first_name": "John",
    "gender": "male",
    "hashed_password": "string",
    "last_name": "Doe",
    "phone_number": {
        "string": "0123123122",
        "valid": False,
    },
    "user_id": 0,
    "username": f"{citizen_id}",
}

passenger = {
    "booking_id": 1,
    "citizen_id": {
        "string": "string",
        "valid": False
    },
    "dob": "2024-12-12T02:10:20+07:00",
    "email": {
        "string": "string",
        "valid": False
    },
    "first_name": "string",
    "gender": "string",
    "last_name": "string",
    "passenger_id": 0,
    "phone_number": {
        "string": "string",
        "valid": False
    },
    "seat_number": "string"
}

citizen_id += 1

def post_stuff(url, data):
    res = s.post(url, json = data)
    print(res.text)

def get_stuff(url):
    res = s.get(url)
    print(res.text)

def put_stuff(url, data):
    res = s.put(url, json = data)
    print(res.text)

def delete_stuff(url):
    res = s.delete(url)
    print(res.text)

post_stuff(aircrafts_url, aircraft)
post_stuff(flights_url, flight)
post_stuff(users_url, user)
post_stuff(bookings_url, booking)
post_stuff(passengers_url, passenger)

{
    "aircraft_id": 9,
    "aircraft_manufacturer": "boeing",
    "aircraft_type": "passenger",
    "aircraft_name": "747",
    "capacity": 500
}
{
    "flight_id": 9,
    "flight_number": "VN123",
    "aircraft_id": 1,
    "departure_airport": "HAN",
    "arrival_airport": "CXR",
    "departure_time": "2024-12-12T02:10:20+07:00",
    "arrival_time": "2024-12-12T02:10:20+07:00",
    "status": "on-time",
    "gate": "A1",
    "board_time": "2024-12-12T02:10:20+07:00"
}
{
    "user_id": 9,
    "citizen_id": "9",
    "username": "9",
    "hashed_password": "string",
    "first_name": "John",
    "last_name": "Doe",
    "email": {
        "String": "string",
        "Valid": true
    },
    "phone_number": {
        "String": "0123123122",
        "Valid": false
    },
    "dob": "2024-12-12T02:10:20+07:00",
    "gender": "male"
}
{
    "booking_id": 9,
    "flight_id": 1,
    "citizen_id": "0123",
    "total_passengers": 20,
    "booking_date": "2024-12-12T02:10:20+07:00",
    "seat_numb

In [46]:
get_stuff(aircrafts_url)
get_stuff(flights_url)
get_stuff(users_url)
get_stuff(bookings_url)
get_stuff(passengers_url)

[
    {
        "aircraft_id": 1,
        "aircraft_manufacturer": "boeing",
        "aircraft_type": "passenger",
        "aircraft_name": "747",
        "capacity": 500
    },
    {
        "aircraft_id": 2,
        "aircraft_manufacturer": "boeing",
        "aircraft_type": "passenger",
        "aircraft_name": "747",
        "capacity": 500
    },
    {
        "aircraft_id": 3,
        "aircraft_manufacturer": "boeing",
        "aircraft_type": "passenger",
        "aircraft_name": "747",
        "capacity": 500
    },
    {
        "aircraft_id": 4,
        "aircraft_manufacturer": "boeing",
        "aircraft_type": "passenger",
        "aircraft_name": "747",
        "capacity": 500
    },
    {
        "aircraft_id": 5,
        "aircraft_manufacturer": "boeing",
        "aircraft_type": "passenger",
        "aircraft_name": "747",
        "capacity": 500
    },
    {
        "aircraft_id": 6,
        "aircraft_manufacturer": "boeing",
        "aircraft_type": "passenger",
     

In [37]:
for i in range(200):
    r = s.get(flights_url)